This is a starter notebook for the project, you'll have to import the libraries you'll need, you can find a list of the ones available in this workspace in the requirements.txt file in this workspace. 

In [1]:
#Step 1: Setting Up the Python Application

In [2]:
import os

os.environ["OPENAI_API_KEY"] = "voc-10609208041266773387460669f72809407a3.29559041"
os.environ["OPENAI_API_BASE"] = "https://openai.vocareum.com/v1"

from langchain.llms import OpenAI


In [3]:
#!pip install -r requirements.txt
!pip install pandas

Defaulting to user installation because normal site-packages is not writeable


In [4]:
import openai
import os

import pandas as pd

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders.csv_loader import CSVLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.llms import OpenAI
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain import LLMChain
from langchain.chains.question_answering import load_qa_chain


In [5]:
#Step 2: Generating Real Estate Listings

In [6]:
MODEL_NAME = "gpt-4o"
API_KEY = "voc-10609208041266773387460669f72809407a3.29559041"

model = OpenAI(model_name=MODEL_NAME, temperature=0, api_key=API_KEY)

prompt="""
Create 10 real estate listings given the example below. Seperate each listing with a "!!!"
Neighborhood: Green Oaks
Price: $800,000
Bedrooms: 3
Bathrooms: 2
House Size: 2,000 sqft

Description: Welcome to this eco-friendly oasis nestled in the heart of Green Oaks. This charming 3-bedroom, 2-bathroom home boasts energy-efficient features such as solar panels and a well-insulated structure. Natural light floods the living spaces, highlighting the beautiful hardwood floors and eco-conscious finishes. The open-concept kitchen and dining area lead to a spacious backyard with a vegetable garden, perfect for the eco-conscious family. Embrace sustainable living without compromising on style in this Green Oaks gem.

Neighborhood Description: Green Oaks is a close-knit, environmentally-conscious community with access to organic grocery stores, community gardens, and bike paths. Take a stroll through the nearby Green Oaks Park or grab a cup of coffee at the cozy Green Bean Cafe. With easy access to public transportation and bike lanes, commuting is a breeze.
"""

/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:202: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/langchain/llms/openai.py:790: UserWarning: You are trying to use a chat model. This way of initializing it is no longer supported. Instead, please use: `from langchain.chat_models import ChatOpenAI`
  warnings.warn(


In [7]:
'''
listings = model(prompt)
listings
listings = listings.split("!!!")
df = pd.DataFrame(listings, columns=['Listings'])
df.to_csv('homes.csv', index=False)
'''

'\nlistings = model(prompt)\nlistings\nlistings = listings.split("!!!")\ndf = pd.DataFrame(listings, columns=[\'Listings\'])\ndf.to_csv(\'homes.csv\', index=False)\n'

In [8]:
df_loaded = pd.read_csv('listings.csv')
print(df_loaded)

                                             Listings
0   Neighborhood: Maplewood Heights\nPrice: $950,0...
1   Neighborhood: Sunnyvale\nPrice: $720,000\nBedr...
2   Neighborhood: Riverwood Estates\nPrice: $1,200...
3   Neighborhood: Willow Creek\nPrice: $650,000\nB...
4   Neighborhood: Pine Ridge\nPrice: $780,000\nBed...
5   Neighborhood: Cedar Grove\nPrice: $890,000\nBe...
6   Neighborhood: Oakwood Hills\nPrice: $1,050,000...
7   Neighborhood: Birchwood Park\nPrice: $700,000\...
8   Neighborhood: Elmwood Village\nPrice: $820,000...
9   Neighborhood: Redwood Meadows\nPrice: $980,000...
10  Neighborhood: Aspen Ridge\nPrice: $1,150,000\n...


In [9]:
#Step 3: Storing Listings in a Vector Database

In [11]:
loader = CSVLoader(file_path='./listings.csv')
docs = loader.load()

In [12]:
splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
split_docs = splitter.split_documents(docs)

In [13]:
embeddings = OpenAIEmbeddings()

db = Chroma.from_documents(split_docs, embeddings)

In [14]:
#Step 4: Building the User Preference Interface

In [15]:
questions = [   
                "How big do you want your house to be?",
                "What are 3 most important things for you in choosing this property?", 
                "Which amenities would you like?", 
                "Which transportation options are important to you?",
                "How urban do you want your neighborhood to be?",   
            ]
answers = [
    "I want a small house for me and my dog",
    "A nice view, good local food, and only one floor.",
    "A backyard for my dog, a parking spot, and a large enough kitchen area for friends and family.",
    "Walkable areas, access to at least one highway",
    "Urban with a enough trees to not feel away from nature"]

In [23]:
prefs = ""
for q, a in zip(questions, answers):
    prefs += (q + " " + a + "\n")

How big do you want your house to be? I want a small house for me and my dog
What are 3 most important things for you in choosing this property? A nice view, good local food, and only one floor.
Which amenities would you like? A backyard for my dog, a parking spot, and a large enough kitchen area for friends and family.
Which transportation options are important to you? Walkable areas, access to at least one highway
How urban do you want your neighborhood to be? Urban with a enough trees to not feel away from nature



In [24]:
#Step 5: Searching based on preferences

In [26]:
similar_docs = db.similarity_search(prefs, k=5)
similar_docs

[Document(page_content="Listings: Neighborhood: Birchwood Park\nPrice: $700,000\nBedrooms: 3\nBathrooms: 2\nHouse Size: 2,000 sqft\n\nDescription: Welcome to this charming 3-bedroom, 2-bathroom home in Birchwood Park. The open-concept living area features a cozy fireplace and hardwood floors. The kitchen is a chef's dream with granite countertops, stainless steel appliances, and a breakfast nook. The master suite includes a walk-in closet and a luxurious bathroom with a soaking tub. The backyard is perfect for entertaining with a large deck and mature trees providing shade and privacy.\n\nNeighborhood Description: Birchwood Park is a picturesque community with tree-lined streets, parks, and excellent schools. Enjoy the local shops, cafes, and restaurants, as well as easy access to hiking and biking trails. Birchwood Park offers a perfect blend of suburban tranquility and urban convenience.", metadata={'row': 7, 'source': './listings.csv'}),
 Document(page_content="Listings: Neighborhoo

In [32]:
#Step 6: Personalizing Listing Descriptions

In [38]:
TEMPLATE = """
Based on the context:

{context}

Choose one listing that best fits the users preferences: {prefs}. Generate a response that lists the neighborhood, price, beds, baths, and size. 
Then create a small summary of the propery that is tailored to the users prefrences 

"""

In [39]:
use_chain_helper = False
if use_chain_helper:
    rag = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=db.as_retriever())
    print(rag.run(query))
else:
    prompt = PromptTemplate(
        template=TEMPLATE,
        input_variables=["prefs", "context"],
    )
    chain = load_qa_chain(model, prompt = prompt, chain_type="stuff")
    print(chain.run(input_documents=similar_docs, prefs = prefs))

**Neighborhood:** Sunnyvale  
**Price:** $720,000  
**Bedrooms:** 3  
**Bathrooms:** 2  
**House Size:** 1,800 sqft  

**Summary:**  
This delightful 3-bedroom, 2-bathroom home in the heart of Sunnyvale is perfect for you and your dog. The single-story layout ensures easy accessibility, and the bright and airy living room with vaulted ceilings and large windows offers a beautiful view and plenty of natural light. The updated kitchen with quartz countertops and stainless steel appliances is spacious enough for hosting friends and family. The backyard is a private oasis with a covered patio, ideal for outdoor dining and relaxation, and provides ample space for your dog to play. 

Sunnyvale is an urban neighborhood with a friendly atmosphere, top-rated schools, parks, and recreational facilities. You'll enjoy the local farmers' market, boutique shops, and a variety of dining options, ensuring you have access to good local food. The area is walkable and offers easy access to at least one h